# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
# To run this from local machine please use "from api_keys import g_key" 
# not from config as config is not loaded for security reasons
# from api_keys import g_key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
city_data = pd.read_csv("cities.csv")
city_data


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Rikitea,13,PF,1623605439,76,-23.1203,-134.9692,74.95,10.07
1,Hilo,40,US,1623605459,77,19.7297,-155.0900,75.61,6.91
2,Quang Ngai,98,VN,1623605459,75,15.1167,108.8000,79.30,4.45
3,Mataura,100,NZ,1623605460,97,-46.1927,168.8643,50.85,4.56
4,Vaini,40,TO,1623605460,83,-21.2000,-175.2000,71.76,8.05
...,...,...,...,...,...,...,...,...,...
558,Ust-Maya,46,RU,1623605682,96,60.4167,134.5333,41.41,3.04
559,Mnogovershinnyy,98,RU,1623605682,62,53.9353,139.9242,37.51,1.16
560,Krasnosel'kup,74,RU,1623605683,56,65.7000,82.4667,50.74,9.60
561,Goderich,85,CA,1623605683,77,43.7501,-81.7165,78.03,10.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"]

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
clean_city_df = city_data.loc[(city_data["Wind Speed"] <= 10) & (city_data["Cloudiness"] == 0) & \
                                   (city_data["Max Temp"] >= 70) & (city_data["Max Temp"] <= 80)].dropna()

clean_city_df.head(10)

#clean_city_df.count()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
85,Mar del Plata,0,AR,1623605373,75,-38.0023,-57.5575,70.09,10.00
97,Jardim,0,BR,1623605496,35,-21.4803,-56.1381,78.51,5.08
129,Fukuechō,0,JP,1623605509,87,32.6881,128.8419,70.45,1.81
245,Tūkrah,0,LY,1623605557,74,32.5341,20.5791,72.81,8.50
267,Sirte,0,LY,1623605401,70,31.2089,16.5887,73.44,8.52
352,Abu Samrah,0,SY,1623605599,23,35.3029,37.1841,77.18,8.57
407,Samarkand,0,UZ,1623605622,25,39.6542,66.9597,76.42,0.00
545,Saint-Pierre,0,RE,1623605677,78,-21.3393,55.4781,73.04,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
# Store variable named hotel_df
hotel_df = clean_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
85,Mar del Plata,AR,-38.0023,-57.5575,
97,Jardim,BR,-21.4803,-56.1381,
129,Fukuechō,JP,32.6881,128.8419,
245,Tūkrah,LY,32.5341,20.5791,
267,Sirte,LY,31.2089,16.5887,
352,Abu Samrah,SY,35.3029,37.1841,
407,Samarkand,UZ,39.6542,66.9597,
545,Saint-Pierre,RE,-21.3393,55.4781,


In [38]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 85: Mar del Plata.
Closest hotel in Mar del Plata is Hotel Las Rocas.
------------
Retrieving Results for Index 97: Jardim.
Closest hotel in Jardim is Hotel Jardim.
------------
Retrieving Results for Index 129: Fukuechō.
Closest hotel in Fukuechō is Hotel Sou.
------------
Retrieving Results for Index 245: Tūkrah.
Closest hotel in Tūkrah is مصيف مرحبا للعائلات.
------------
Retrieving Results for Index 267: Sirte.
Closest hotel in Sirte is Al- Waffaa Hotel.
------------
Retrieving Results for Index 352: Abu Samrah.
Missing field/result... skipping.
------------
Retrieving Results for Index 407: Samarkand.
Closest hotel in Samarkand is Hotel DiliMah Premium Luxury.
------------
Retrieving Results for Index 545: Saint-Pierre.
Closest hotel in Saint-Pierre is Villa Delisle, Hotel & Spa.
------------
-------End of Search-------


In [39]:
hotel_df


,City,Country,Lat,Lng,Hotel Name
85,Mar del Plata,AR,-38.0023,-57.5575,Hotel Las Rocas
97,Jardim,BR,-21.4803,-56.1381,Hotel Jardim
129,Fukuechō,JP,32.6881,128.8419,Hotel Sou
245,Tūkrah,LY,32.5341,20.5791,مصيف مرحبا للعائلات
267,Sirte,LY,31.2089,16.5887,Al- Waffaa Hotel
352,Abu Samrah,SY,35.3029,37.1841,
407,Samarkand,UZ,39.6542,66.9597,Hotel DiliMah Premium Luxury
545,Saint-Pierre,RE,-21.3393,55.4781,"Villa Delisle, Hotel & Spa"


In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))